In [2]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.metrics import log_loss


In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
y=train['OUTCOME']
ID = test['ID']
data=pd.concat((train,test)).reset_index(drop=True).copy()
data=data.drop(columns=['ID','OUTCOME'],axis=1)

In [4]:
cat_features = [c for c, dt in data.dtypes.items() if dt=='object']
data[cat_features] = data[cat_features].astype("category")

In [5]:
train=data[:len(train)]
test=data[len(train):]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_X, val_X, train_y, val_y = train_test_split(train, y, test_size=0.33, random_state=42)

In [10]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.25-b02, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Ayush\AppData\Local\Temp\tmp6x68syrf
  JVM stdout: C:\Users\Ayush\AppData\Local\Temp\tmp6x68syrf\h2o_Ayush_started_from_python.out
  JVM stderr: C:\Users\Ayush\AppData\Local\Temp\tmp6x68syrf\h2o_Ayush_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 year and 14 days !!!
H2O_cluster_name:,H2O_from_python_Ayush_0zorya
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.420 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [14]:
training = pd.concat([train,y],axis=1)

In [16]:
hf_train = h2o.H2OFrame(training)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [18]:
y='OUTCOME'
x=hf_train.columns
x.remove(y)

In [20]:
aml = H2OAutoML(max_models=20,seed=1)
aml.train(x=x,y=y,training_frame=hf_train)

AutoML progress: |
10:44:13.302: AutoML: XGBoost is not available; skipping it.
10:44:13.303: Step 'best_of_family_xgboost' not defined in provider 'StackedEnsemble': skipping it.
10:44:13.303: Step 'all_xgboost' not defined in provider 'StackedEnsemble': skipping it.

██████████████████████████████████████████████████████████████ (cancelled)  99%


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_bed7beaf35c5dd4457bda8bda8404c0f> was cancelled by the user.

In [ ]:
lb = aml.leaderboard
lb

In [ ]:
hf_test = h2o.H2OFrame(test)

In [ ]:
preds = aml.predict(hf_test)

In [ ]:
df = hf_test.cbind(preds)
df.head(2)